In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models


ModuleNotFoundError: No module named 'pandas'

In [ ]:
zip_ref = zipfile.ZipFile('./dataset/data_balita.zip', 'r')
zip_ref.extractall('./dataset/data_balita')
zip_ref.close()



: 

In [ ]:
data = pd.read_csv('./dataset/data_balita/data_balita.csv')
data

: 

In [ ]:
data['Jenis Kelamin'] = data['Jenis Kelamin'].map({'laki-laki': 0, 'perempuan': 1})
data['Status Gizi'] = data['Status Gizi'].map({'severely stunted': 0, 'stunted': 1, 'normal': 2, 'tinggi': 3})

data

: 

In [ ]:
data_train = data.sample(frac=0.7, random_state=18)
data_temp = data.drop(data_train.index)
data_val = data_temp.sample(frac=0.7, random_state=18)
data_test = data_temp.drop(data_val.index)

: 

In [ ]:
dt_train=data_train.drop('Status Gizi', axis=1)

dt_train

: 

In [ ]:
label_train = data_train['Status Gizi']

label_train

: 

In [ ]:
dt_val = data_val.drop('Status Gizi', axis=1)
dt_val

: 

In [ ]:
label_val = data_val['Status Gizi']

label_val

: 

In [ ]:
def lr_scheduler(epoch):
    # Menghitung kecepatan pembelajaran berdasarkan epoch
    return 0.001 * np.exp(-0.1 * epoch)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True
)

: 

In [ ]:
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(dt_train.shape[1],)),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.CategoricalAccuracy()])
history_model = model.fit(dt_train, label_train, epochs=50, validation_data=(dt_val, label_val),callbacks=[lr_callback, callback])

: 

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(10, 5))
plt.title('Model Accuracy')

plt.plot(history_model.history['accuracy'], label='Accuracy (Train)')
plt.plot(history_model.history['val_accuracy'], label='Accuracy (Validation)')
plt.plot(history_model.history['categorical_accuracy'], label='Categorical Accuracy (Train)')
plt.plot(history_model.history['val_categorical_accuracy'], label='Categorical Accuracy (Validation)')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.ylim([0, 1.01])

plt.legend(loc='lower right')

test_acc_model = model.evaluate(dt_val, label_val, verbose=2)

plt.show()

: 

In [ ]:
plt.plot(history_model.history['loss'], label='Training Loss')
plt.plot(history_model.history['val_loss'], label='Validation Loss')

plt.title('Model Loss')

# Menambahkan label sumbu x dan y
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Menampilkan legenda
plt.legend(['Train', 'Test'], loc='upper left')

# Menampilkan plot
plt.show()

: 

In [ ]:
dt_test = data_test.drop('Status Gizi', axis=1)

label_test = data_test['Status Gizi']

: 

In [ ]:
def toClass(pred):
    class_ = np.zeros(len(pred))

    for i in range(len(pred)):
        index = pred[i].argmax()
        class_[i] = index
    return class_

: 

In [ ]:

label_pred = model.predict(dt_test)

cm = confusion_matrix(label_test, toClass(label_pred))
label=['severely stunted', 'stunted', 'normal', 'tinggi']
df1 = pd.DataFrame(columns=label, index=label, data=cm)

f, ax = plt.subplots(figsize=(6,6))
sns.heatmap(df1, annot=True, cmap="Greens", fmt='.0f', ax=ax, linewidths=5, cbar=False, annot_kws={"size": 16})

plt.xlabel("Predicted Label")
plt.xticks(size=12)
plt.yticks(size=12, rotation=0)
plt.ylabel("True Label")
plt.title("Confusion Matrix", size=12)

plt.show()

accuracy = accuracy_score(label_test, toClass(label_pred))
print('\n Tingkat Akurasi Prediksi:', accuracy)

: 

In [ ]:
model.save('./model/stunting_prediction.h5')

: 